# First Machine Learning Model

Sarah Braverman

In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


#### When done, put the imputing and OHE code here to be able to create and use a model

**Model Purpose:** 
**A model to predict whether the avalanche will hit the rail depending on the conditions** 

List: 

Do OHE, imputing etc. 

Test parameters, what are the best settings? Max_depth, n_estimators etc.

Figure out how to find out which features impact the rail the most
 
Train model on train data, fit on all data, train on all data, see how accurate it can be

#### MG's Code is below, OHE is done, does not have imputing. 

In [3]:
import numpy as np

avy_obs = pd.read_pickle('data-prep/pkl/avy_obs.pkl')

In [4]:
X = avy_obs.drop('deb_rail', axis=1)
y = avy_obs['deb_rail']
X = X.drop(['avcomment', 'edcr_user', 'edcr_ip',
       'eded_user', 'eded_time', 'eded_ip'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 1)
cols_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
cols_missing_num = [col for col in X_train.columns if (X_train[col].dtypes != object) & (col in cols_missing)]

cols_missing_num
cols_missing_cat = [col for col in X_train.columns if (X_train[col].dtypes == object) & (col in cols_missing)]

cols_missing_cat
object_nunique = list(map(lambda col: X_train[col].nunique(), cols_missing_cat))
d = dict(zip(cols_missing_cat, object_nunique))

sorted(d.items(), key=lambda x: x[1])
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[cols_missing_cat]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[cols_missing_cat]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

OH_cols_train.columns = OH_encoder.get_feature_names_out()
OH_cols_valid.columns = OH_encoder.get_feature_names_out()

num_X_train = X_train.drop(cols_missing_cat, axis=1)
num_X_valid = X_valid.drop(cols_missing_cat, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)